In [94]:
import requests  # pip install requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import statistics as stat
from functools import reduce

date = '12-28-2020'
home_id = 'ATL'
away_id = 'DET'

In [95]:
date_triple = date.split('-')
month = date_triple[0]
day = date_triple[1]
year = date_triple[2]
url = 'https://www.basketball-reference.com/boxscores/' + year + month + day + '0' + home_id + '.html'
ids = [away_id, home_id]

In [96]:
# basic boxscore - away
df_away = pd.read_html(url)[0]
df_away = pd.concat([df_away['Unnamed: 0_level_0'], df_away['Basic Box Score Stats']], axis=1, sort=False)
#advanced boxscore - away
df_away_adv = pd.read_html(url)[7]
# clean up advanced boxscore - away
df_away_adv1 = pd.concat([df_away_adv['Unnamed: 0_level_0'],df_away_adv['Advanced Box Score Stats']], axis=1, sort=False)
df_away_adv = df_away_adv1.drop(['Starters','MP'],axis=1)

# basic boxscore - home
df_home = pd.read_html(url)[8]
df_home = pd.concat([df_home['Unnamed: 0_level_0'], df_home['Basic Box Score Stats']], axis=1, sort=False)
# advanced boxscore - home
df_home_adv = pd.read_html(url)[15]
# clean up advanced boxscore - home
df_home_adv1 = pd.concat([df_home_adv['Unnamed: 0_level_0'],df_home_adv['Advanced Box Score Stats']], axis=1, sort=False)
df_home_adv = df_home_adv1.drop(['Starters','MP'],axis=1)


In [97]:
df_away_all = pd.concat([df_away, df_away_adv], axis=1, sort=False)
df_home_all = pd.concat([df_home, df_home_adv], axis=1, sort=False)

df_list = [df_away_all, df_home_all]

In [98]:
for home_away in range(2):
        df = df_list[home_away]

        # Deletes unwanted rows and joins 2 sub-dataframes into a single dataframe
        rows = list(df.index.values)
        last_row = rows[len(rows) - 1]
        df = df.drop([5, last_row])
        #df = pd.concat([df['Unnamed: 0_level_0'], df['Basic Box Score Stats'],df['Advanced Box Score Stats']],  axis=1, sort=False)


In [104]:
pd.set_option('display.max_columns', None)

In [108]:
for home_away in range(2):
        df = df_list[home_away]

        # Deletes unwanted rows and joins 2 sub-dataframes into a single dataframe
        rows = list(df.index.values)
        last_row = rows[len(rows) - 1]
        df = df.drop([5, last_row])

        # Creates and adds a "status" variable (1 if starter, 0 if reserve)
        starter = [0] * df.shape[0]
        for i in range(5):
            starter[i] = 1
        df.insert(1, 'Status', starter)

        # Creates and adds a "Team" (Team ID) and "Home" (Home team or Away team)
        home = [home_away] * df.shape[0]
        team = [ids[home_away]] * df.shape[0]
        df.insert(1, 'Team', team)
        df.insert(2, 'Home', home)

        # Changes "MP" time format from string '00:00' to float 0.0.
        # Did not play will appear as a 2 in status column. Did not dress will appear as 3
        rows = list(df.index.values)
        for i in rows:
            if df['MP'][i] == 'Did Not Play':
                df['Status'][i] = 2
            elif df['MP'][i] == 'Did Not Dress':
                df['Status'][i] = 3
            else:
                col_index = df['MP'][i].find(':')
                df['MP'][i] = int(df['MP'][i][:col_index]) + int(df['MP'][i][(col_index+1):])/60
         
        # Replace Did not play and did not dress with zeros
        df = df.replace('Did Not Play', 0)
        df = df.replace('Did Not Dress',0)
        
        print(df)
                
        # Changes values in the "+/-" column from strings to integers
        rows = list(df.index.values)
        for i in rows:
            if df['+/-'][i] == '0':
                df['+/-'][i] = 0
            elif df['+/-'][i] == 0:
                df['+/-'][i] = 0
            elif df['+/-'][i][0] == '+':
                df['+/-'][i] = int(df['+/-'][i][1:])
            else:
                df['+/-'][i] = -1 * int(df['+/-'][i][1:])

        # Drops any column that relates to efficiency stats
        df.drop(['FG%', '3P%', 'FT%'], inplace=True, axis=1)
        
        # Add fantasy points column
        #fp = []
        #for i in rows:
        #    fp[i] = df["PTS"] + (1.2*df["TRB"]) + (1.5*df["AST"]) + (3*df["STL"]) + (3*df["BLK"]) - df["TOV"]
        #df.insert(-1,'Fantasy Points',fp)

        print(df)

                 Starters Team  Home  Status         MP  FG FGA    FG% 3P 3PA  \
0            Jerami Grant  DET     0       1  37.633333   9  20   .450  3   8   
1           Mason Plumlee  DET     0       1  33.550000   4   4  1.000  0   0   
2              Saddiq Bey  DET     0       1  30.250000   3  11   .273  2   7   
3           Killian Hayes  DET     0       1  29.383333   4  12   .333  2   6   
4            Josh Jackson  DET     0       1  28.016667  11  19   .579  3   7   
6            Delon Wright  DET     0       0  21.750000   5   9   .556  1   2   
7         Wayne Ellington  DET     0       0  20.550000   4  10   .400  2   7   
8          Isaiah Stewart  DET     0       0  14.450000   1   4   .250  0   0   
9   Sviatoslav Mykhailiuk  DET     0       0  14.050000   1   6   .167  0   4   
10        Sekou Doumbouya  DET     0       0  10.366667   1   3   .333  0   0   
11      Deividas Sirvydis  DET     0       2   0.000000   0   0      0  0   0   
12              Saben Lee  D

/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipyke